In [17]:
setwd("/Users/rebecca/sudmant/analyses/myotis/analysis/exploratory/species_peaks")

library(GSA)
library(dplyr)
library(qvalue)
library(ggplot2)
library(data.table)

source("/Users/rebecca/sudmant/analyses/myotis/code/fisher_test.R")

shared_genes <- fread("results/data/shared_genes.csv", data.table = FALSE)

In [ ]:
## Load in Broad gene sets:
gsc <- GSA.read.gmt("/Users/rebecca/sudmant/analyses/myotis/data/genesets/msigdb_v2023.2.Hs_GMTs/msigdb.v2023.2.Hs.symbols.gmt")
sets <- gsc$genesets
names(sets) <- gsc$geneset.names
sets <- sets[grep("^REACTOME|^GOBP", names(sets))]

## Shared peaks

In [19]:
shared_peaks <- fread("results/data/shared_promoters_shared_peaks.csv", data.table = FALSE)
shared_peak_enrichments <- sort(unlist(lapply(sets, fisher_test,
                                              mod = shared_peaks$Gene,
                                              all = shared_genes$Gene)))
shared_peak_enrich_df <- data.frame(Set = names(shared_peak_enrichments), 
                                    Pval = shared_peak_enrichments,
                                    row.names = NULL)
shared_peak_enrich_df$Qval <- qvalue(shared_peak_enrich_df$Pval)$qvalues
head(shared_peak_enrich_df, 15)

,Set,Pval,Qval
,<chr>,<dbl>,<dbl>
1,REACTOME_DNA_DAMAGE_TELOMERE_STRESS_INDUCED_SENESCENCE,0.0001047510,0.9782699
2,REACTOME_COPI_MEDIATED_ANTEROGRADE_TRANSPORT,0.0003555364,1.0000000
3,REACTOME_COPI_DEPENDENT_GOLGI_TO_ER_RETROGRADE_TRAFFIC,0.0005960440,1.0000000
4,REACTOME_RECOGNITION_AND_ASSOCIATION_OF_DNA_GLYCOSYLASE_WITH_SITE_CONTAINING_AN_AFFECTED_PURINE,0.0006966203,1.0000000
5,REACTOME_HATS_ACETYLATE_HISTONES,0.0009213978,1.0000000
6,REACTOME_BASE_EXCISION_REPAIR_AP_SITE_FORMATION,0.0011013265,1.0000000
7,REACTOME_GOLGI_TO_ER_RETROGRADE_TRANSPORT,0.0012272583,1.0000000
8,REACTOME_INHIBITION_OF_DNA_RECOMBINATION_AT_TELOMERE,0.0012560298,1.0000000
9,REACTOME_ER_TO_GOLGI_ANTEROGRADE_TRANSPORT,0.0016569590,1.0000000


In [11]:
shared_peaks$Gene

[1] "ARF1"    "CASP1"   "CCDC88C" "CEL"     "CES1"    "CREBBP"  "CYP4V2" 
 [8] "DPF3"    "FBXL2"   "FER1L6"  "H1-3"    "H4C2"    "HINT3"   "ING4"   
[15] "KDELR2"  "LY6H"    "LYNX1"   "MSX2"    "NME4"    "NSUN3"   "PRX"    
[22] "PTPMT1"  "RPL32"   "SASH1"   "SBSPON"  "SCAND1"  "SEMA4F"  "SMPD3"  
[29] "SNRNP27" "SP140L"  "SYNGR1"  "TINF2"   "TUBA1B"  "UTP14A"  "ZNF771"

### No significant enrichments in shared peaks

## Semi-shared peaks

In [20]:
semi_shared_peaks <- fread("results/data/shared_promoters_semi_shared_peaks.csv", data.table = FALSE)

species <- unique(semi_shared_peaks$Species)
                                                                                 
missing_peak_enrichments <- lapply(species, function(spec){
  
  missing_peaks_spec <- semi_shared_peaks[semi_shared_peaks$Species %in% spec,]
  missing_peaks_enrich <- sort(unlist(lapply(sets, fisher_test,
                                             mod = missing_peaks_spec$Gene,
                                             all = shared_genes$Gene)))
  return(data.frame(Species = spec,
                    Set = names(missing_peaks_enrich), 
                    Pval = missing_peaks_enrich,
                    Qval = qvalue(missing_peaks_enrich)$qvalues,
                    row.names = NULL))
  
})
names(missing_peak_enrichments) <- species

In [14]:
tapply(semi_shared_peaks$Gene, semi_shared_peaks$Species, "[")

$`Myotis-Auriculus`
[1] "COX8A" "RPLP1"

$`Myotis-Californicus`
 [1] "ADAM9"    "CFHR4"    "IRF4"     "NPHS1"    "PDPR"     "PPIA"    
 [7] "PPP4R1"   "PRPS2"    "RRBP1"    "S100A7"   "SLC25A24" "TMED9"   

$`Myotis-Evotis`
[1] "CTRB1"    "DDX54"    "POLR2B"   "SERPINB1" "ZNF296"  

$`Myotis-Lucifugus`
[1] "ACTL7B" "CCT5"   "CD46"   "H2AC19"

$`Myotis-Occultus`
[1] "ATP5MC2" "MROH1"   "SETMAR"  "SLC35G1"

$`Myotis-Thysanodes`
[1] "ALOX12"  "DOC2B"   "PLK3"    "PPP1R2"  "TFCP2L1" "TRIM47" 

$`Myotis-Velifer`
[1] "VNN1"

$`Myotis-Volans`
[1] "CBR1"   "CCNYL1" "FABP3"  "MKRN1"  "TAAR1"  "TSSK1B"

$`Myotis-Yumanensis`
[1] "GNG5"     "GPKOW"    "HERC5"    "HSPA1A"   "IRGC"     "RAP2C"    "SLBP"    
[8] "SLC7A6OS" "SPEF1"

In [21]:
lapply(missing_peak_enrichments, head, 10)

,Species,Set,Pval,Qval
,<chr>,<chr>,<dbl>,<dbl>
1,Myotis-Lucifugus,GOBP_SINGLE_FERTILIZATION,0.0004596155,1
2,Myotis-Lucifugus,GOBP_FERTILIZATION,0.0006662110,1
3,Myotis-Lucifugus,GOBP_POSITIVE_REGULATION_OF_MEMORY_T_CELL_DIFFERENTIATION,0.0019022251,1
4,Myotis-Lucifugus,REACTOME_DNA_METHYLATION,0.0026623547,1
5,Myotis-Lucifugus,REACTOME_FOLDING_OF_ACTIN_BY_CCT_TRIC,0.0026623547,1
6,Myotis-Lucifugus,GOBP_IMMUNOLOGICAL_MEMORY_FORMATION_PROCESS,0.0026623547,1
7,Myotis-Lucifugus,GOBP_NEGATIVE_REGULATION_OF_COMPLEMENT_ACTIVATION,0.0026623547,1
8,Myotis-Lucifugus,GOBP_NEGATIVE_REGULATION_OF_HUMORAL_IMMUNE_RESPONSE_MEDIATED_BY_CIRCULATING_IMMUNOGLOBULIN,0.0026623547,1
9,Myotis-Lucifugus,GOBP_POSITIVE_REGULATION_OF_ESTABLISHMENT_OF_PROTEIN_LOCALIZATION_TO_TELOMERE,0.0026623547,1


### No significant enrichments in N-1 peaks

## Private peaks

In [22]:
private_peaks <- fread("results/data/shared_promoters_private_peaks.csv", data.table = FALSE)

species <- unique(private_peaks$Species)
                                                                             
private_peak_enrichments <- lapply(species, function(spec){
  
  private_peaks_spec <- private_peaks[private_peaks$Species %in% spec,]
  private_peaks_enrich <- sort(unlist(lapply(sets, fisher_test,
                                             mod = private_peaks_spec$Gene,
                                             all = shared_genes$Gene)))
  return(data.frame(Species = spec,
                    Set = names(private_peaks_enrich), 
                    Pval = private_peaks_enrich,
                    Qval = qvalue(private_peaks_enrich)$qvalues,
                    row.names = NULL))
  
})
names(private_peak_enrichments) <- species

In [6]:
tapply(private_peaks$Gene, private_peaks$Species, "[")

$`Myotis-Auriculus`
 [1] "ADH6"    "GK"      "GOLPH3"  "H2AZ1"   "HMX3"    "KLK9"    "OR51V1" 
 [8] "PABPC4"  "PLP2"    "RUVBL1"  "STRA6"   "TMEM127" "ZC3H3"  

$`Myotis-Californicus`
[1] "ARFGEF1" "EIF4A3" 

$`Myotis-Evotis`
 [1] "CLXN"      "CP"        "EIF3G"     "KRTAP13-2" "OR51A7"    "OR51C1P"  
 [7] "PKM"       "PRKRA"     "SIRPA"     "SRP14"     "TOMM22"    "TRUB2"    

$`Myotis-Lucifugus`
 [1] "ATP8B2"    "BSG"       "BTBD18"    "BTNL2"     "CHIA"      "CLCA1"    
 [7] "DDX60"     "DTD2"      "EDC3"      "EIF2S2"    "ENPP7"     "FAU"      
[13] "FLT1"      "GARIN3"    "GPBP1L1"   "GPS2"      "HNRNPH1"   "IVL"      
[19] "MANEAL"    "MCM3AP"    "MED4"      "MIA3"      "MRPS12"    "NEK2"     
[25] "NUP62"     "OR52B2"    "OR56A1"    "PACRG"     "PARP12"    "PRELID3B" 
[31] "RANBP3"    "RELT"      "TAAR2"     "TAF1"      "TARM1"     "TAS2R16"  
[37] "TMPRSS11D" "TRRAP"     "TTC7B"     "ZNF207"   

$`Myotis-Occultus`
 [1] "AKR1D1"  "C4BPA"   "CDK2AP1" "CELA2A"  "CKAP2"   "FAM169A" "FHL1"   
 [8] "MRPS33"  "NCL"     "RHOA"    "TMEM233" "UBQLN1"  "WTAP"    "ZBED6"  

$`Myotis-Thysanodes`
 [1] "ARG2"    "BAAT"    "C1D"     "CYTIP"   "DMTN"    "KIF4A"   "LYPLA1" 
 [8] "MSTN"    "MTMR1"   "NACC1"   "OBSCN"   "PRSS55"  "PRSS8"   "SDAD1"  
[15] "SGF29"   "SINHCAF" "SRSF3"   "UBE2V1" 

$`Myotis-Velifer`
 [1] "ABCA13"    "ALDOA"     "ATP1A1"    "BPNT2"     "CDC34"     "CNBD2"    
 [7] "COMMD1"    "DYNLT1"    "GOLPH3L"   "H2AZ2"     "HINT1"     "ISCU"     
[13] "KLRF1"     "KRTAP16-1" "LAGE3"     "MUC4"      "MYBL2"     "MYH3"     
[19] "NLRP1"     "NR1D2"     "PSMD7"     "PTPRZ1"    "RNF213"    "RPL6"     
[25] "RPS28"     "RPS7"      "RPS8"      "SF3B2"     "SPRYD4"    "TBC1D7"   
[31] "TMED10"    "TMEM199"   "TRIP11"    "UCK2"      "WASHC3"    "WRNIP1"   
[37] "ZBTB8OS"  

$`Myotis-Volans`
 [1] "ADAL"    "ADRM1"   "ANKRD17" "ARSF"    "ATP2B4"  "CACNA1H" "CHFR"   
 [8] "CMA1"    "FADS2"   "FBXO22"  "FPR2"    "FTMT"    "HNRNPDL" "HTR3C"  
[15] "IGBP1"   "MPC1L"   "MTCP1"   "NDUFA3"  "OR52D1"  "RPS29"   "RPS3"   
[22] "SLC6A4"  "TBCB"   

$`Myotis-Yumanensis`
[1] "ADPRHL1" "AOC3"    "KLHL15"  "METTL22" "SARDH"   "ZC3H11A"

In [10]:
lapply(private_peak_enrichments, head, 10)

,Species,Set,Pval,Qval
,<chr>,<chr>,<dbl>,<dbl>
1,Myotis-Velifer,LAKE_ADULT_KIDNEY_C18_COLLECTING_DUCT_PRINCIPAL_CELLS_MEDULLA,1.037684e-08,0.000358520
2,Myotis-Velifer,HSIAO_HOUSEKEEPING_GENES,7.196842e-08,0.001243255
3,Myotis-Velifer,LAKE_ADULT_KIDNEY_C19_COLLECTING_DUCT_INTERCALATED_CELLS_TYPE_A_MEDULLA,1.602782e-07,0.001845870
4,Myotis-Velifer,MORF_NPM1,9.186881e-07,0.007935168
5,Myotis-Velifer,LAKE_ADULT_KIDNEY_C8_DECENDING_THIN_LIMB,1.268707e-06,0.008766767
6,Myotis-Velifer,BUSSLINGER_DUODENAL_TRANSIT_AMPLIFYING_CELLS,2.397945e-06,0.013808168
7,Myotis-Velifer,GCM_NPM1,3.629490e-06,0.017914126
8,Myotis-Velifer,LAKE_ADULT_KIDNEY_C7_PROXIMAL_TUBULE_EPITHELIAL_CELLS_S3,6.303039e-06,0.026205168
9,Myotis-Velifer,MORF_ACTG1,7.165900e-06,0.026205168


In [12]:
names(private_peak_enrichments)
library(stringr)

[1] "Myotis-Velifer"      "Myotis-Volans"       "Myotis-Auriculus"   
[4] "Myotis-Yumanensis"   "Myotis-Occultus"     "Myotis-Californicus"
[7] "Myotis-Thysanodes"   "Myotis-Lucifugus"    "Myotis-Evotis"

In [14]:
head(df)

,Species,Set,Pval,Qval
,<chr>,<chr>,<dbl>,<dbl>
1,Myotis-Velifer,LAKE_ADULT_KIDNEY_C18_COLLECTING_DUCT_PRINCIPAL_CELLS_MEDULLA,1.037684e-08,0.000358520
2,Myotis-Velifer,HSIAO_HOUSEKEEPING_GENES,7.196842e-08,0.001243255
3,Myotis-Velifer,LAKE_ADULT_KIDNEY_C19_COLLECTING_DUCT_INTERCALATED_CELLS_TYPE_A_MEDULLA,1.602782e-07,0.001845870
4,Myotis-Velifer,MORF_NPM1,9.186881e-07,0.007935168
5,Myotis-Velifer,LAKE_ADULT_KIDNEY_C8_DECENDING_THIN_LIMB,1.268707e-06,0.008766767
6,Myotis-Velifer,BUSSLINGER_DUODENAL_TRANSIT_AMPLIFYING_CELLS,2.397945e-06,0.013808168


In [16]:
## Plot signif. enrichments:

df <- private_peak_enrichments[[1]]
df <- df %>%
  dplyr::filter(Qval < .05) %>%
  dplyr::mutate(
    Set = sapply(gsub("_", " ", Set), strwrap, width = 25)
  )
  
ggplot(df, aes(x = Set, y = -log10(Qval))) +
  geom_bar(stat = "identity") +
  theme_minimal() +
    theme(axis.title.x = element_blank(),
          axis.text.x = element_text(angle = 45, hjust = 1),
          axis.title.y = element_text(margin = margin(r = 15)),
          panel.grid = element_blank(),
          plot.margin = unit(c(4, 6, 4, 6), "cm")) +
    labs(title = "Proportion of transposable elements with peaks") +
    ylab("Proportion")

ERROR while rich displaying an object: Error in `geom_bar()`:
! Problem while setting up geom.
i Error occurred in the 1st layer.
Caused by error in `abs()`:
! non-numeric argument to mathematical function

Traceback:
1. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
2. tryCatchList(expr, classes, parentenv, handlers)
3. tryCatchOne(expr, names, parentenv, handlers[[1L]])
4. doTryCatch(return(expr), name, parentenv, handler)
5. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . 

## Semi-private peaks

In [3]:
# tapply(private_peaks$Gene, private_peaks$Species, "[")

In [1]:
# private_peaks <- fread("results/data/shared_promoters_semi_private_peaks.csv", data.table = FALSE)

# all_pairs <- table(private_peaks$Species)

# species <- names(all_pairs)[all_pairs >= 3]

# species <- species                        
# private_peak_enrichments <- lapply(species, function(spec){
  
#   private_peaks_spec <- private_peaks[private_peaks$Species %in% spec,]
#   private_peaks_enrich <- sort(unlist(lapply(sets, fisher_test,
#                                              mod = private_peaks_spec$Gene,
#                                              all = shared_genes$Gene)))
#   return(data.frame(Species = spec,
#                     Set = names(private_peaks_enrich), 
#                     Pval = private_peaks_enrich,
#                     Qval = qvalue(private_peaks_enrich)$qvalues,
#                     row.names = NULL))
  
# })
# names(private_peak_enrichments) <- species

In [2]:
# lapply(private_peak_enrichments, head, 10)